In [18]:
from shapely.geometry import shape
import geopandas as gpd
import json
import folium
import requests

# 定义一个函数，根据数值返回对应的颜色
def color_producer(value):
    if value is None:
        return 'gray'  # 处理缺失值的情况，将颜色设为灰色
    elif value <= 5:
        return 'green'
    elif value <= 10:
        return 'yellow'
    else:
        return 'red'

api_endpoint = "http://127.0.0.1:9090/EPA/air-quality-phn"

# 发送 GET 请求到函数的 API 端点
response_json = requests.get(api_endpoint)

# 检查响应状态码
if response_json.status_code == 200:
    response = response_json.json()  # 直接解析 JSON 响应
    data = response['data']
    for feature in data:
        feature['geometry'] = shape(feature['geometry'])
    gdf = gpd.GeoDataFrame(data, geometry='geometry', crs='EPSG:4326')
    gdf['geometry'] = gdf['geometry'].simplify(tolerance=0.001, preserve_topology=True)
    
    # 创建一个地图对象
    m = folium.Map(location=[gdf['geometry'].centroid.y.mean(), gdf['geometry'].centroid.x.mean()], zoom_start=10)
    
    polution_names = ['Particles', 'PM2.5', 'PM10', 'Visibility', 'API', 'O3', 'NO2', 'SO2', 'CO']

    # 创建一个图层控制对象
    layer_control = folium.LayerControl()

    for polution in polution_names:
        # 创建一个新的地理数据框用于当前污染物
        gdf_polution = gdf[[polution, 'geometry']].copy()

        # 将 GeoDataFrame 转换为 GeoJSON 字符串
        geojson_str = gdf_polution.to_json()

        # 添加 GeoJSON 图层到地图上，并设置样式函数
        folium.GeoJson(
            geojson_str,
            style_function=lambda feature: {
                'fillColor': color_producer(feature['properties'][polution]),
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.6,
            },
            tooltip=folium.GeoJsonTooltip(fields=[polution], aliases=[polution], localize=True),  # 添加工具提示，显示对应污染物数值
            name=polution  # 设置图层名称为污染物名称
        ).add_to(m)

        # 添加文本标签，显示每个区域的具体数值
        for idx, row in gdf_polution.iterrows():
            lon, lat = row['geometry'].centroid.x, row['geometry'].centroid.y
            value = row[polution]
            # 限制小数位数为两位
            value_str = "{:.2f}".format(value)
            folium.Marker(
                [lat, lon],
                popup=f"{polution}: {value_str}",
                icon=folium.DivIcon(html=f"<div style='font-size: 10pt; color: black;'>{value_str}</div>")
            ).add_to(m)

    # 添加图层控制到地图
    layer_control.add_to(m)

    # 保存地图到文件
    m.save('air_quality_map.html')
    print("地图已生成并保存到 air_quality_map.html 文件中")
    display(m)

else:
    print("Error in API request:", response_json.text)


Error in API request: error sending request to function
